In [ ]:
import pandas as pd
import numpy as np

In [ ]:
res_dir = '../Resources/'

In [ ]:
df = pd.read_csv(res_dir + 'team_completed_df.csv')

### prepare for team name encoding

In [ ]:
df2 = df[['team_1','team_2','map_winner']]
df2.shape

In [ ]:
create_name_x = 0
# covers upto thr, for example
# thr = df2.shape[0] / 2
def create_name_1(row):
    global subsum
    create_name_x += row['count']
    if create_name_x < thr:
        return row['team']
    elif row['count'] < 2:
        return 'oThErS'
    else:
        return 'oThErS_' + str(int(row['win_ratio'] * 10))

# support most popular 52 teams.
def create_name_2(row):
    global create_name_x
    create_name_x += 1
    if create_name_x < 55:
        return row['team']
    elif row['count'] < 2:
        return 'oThErS'
    else:
        return 'oThErS_' + str(int(row['win_ratio'] * 10))

def team_df_transform(t_df):
    global create_name_x
    t_df.columns = ['team', 'win']
    t_df = t_df.groupby('team').agg({'win': ['mean', 'count']})
    t_df = t_df.reset_index()
    t_df.columns = ['team', 'win_ratio', 'count']
    t_df = t_df.sort_values(by=['count'], ascending=False)
    #
    create_name_x = 0
    t_df['new_name'] = t_df.apply(lambda row: create_name_2(row), axis='columns')
    return t_df

In [ ]:
# team1 map_win 
t_df = df2[['team_1', 'map_winner']].copy()
arr = np.array(t_df['map_winner'].tolist())
t_df['map_winner'] = list(2 - arr)
#
t1_df = team_df_transform(t_df)
t1_df.head()

In [ ]:
# team2 map_win 
t_df = df2[['team_2', 'map_winner']].copy()
arr = np.array(t_df['map_winner'].tolist())
t_df['map_winner'] = list(arr - 1)
#
t2_df = team_df_transform(t_df)
t2_df

####  merge to teamname encoding df

In [ ]:
team_df = t1_df.merge(t2_df, on='team', how='outer')
team_df.columns = ['team',
                   't1_win_ratio', 't1_count', 't1_name',
                   't2_win_ratio', 't2_count', 't2_name']
team_df.sort_values(by=['team'])

In [ ]:
def_names = {'t1_name': 'oThErS', 't2_name': 'oThErS'}
team_df = team_df.fillna(value=def_names)
team_encode_df = team_df[['team', 't1_name', 't2_name']]
team_encode_df = team_encode_df.set_index('team')
team_encode_df

In [ ]:
supported_teams = team_encode_df.index.tolist()
len(supported_teams)

### encode

In [ ]:
df3 = df2.copy()
df3.columns = ['t1','t2','map_winner']
df3.head(2)

In [ ]:
# convert to X teamname
def create_X_name(row):
    # default unknown
    t1 = t2 = 'oThErS'
    # team_1
    t = row['t1']
    if t in supported_teams:
        t1 = team_encode_df.loc[t, 't1_name']
    # team_2
    t = row['t2']
    if t in supported_teams:
        t2 = team_encode_df.loc[t, 't2_name']
    return [t1, t2]
df3[['team_1', 'team_2']] = df3.apply(lambda row: create_X_name(row), axis='columns', result_type='expand')
df3.head(2)

In [ ]:
df2 = df3[['team_1','team_2','map_winner']]
df2

In [ ]:
df2['team_1'].value_counts()

### done dataframe preparation

In [ ]:
# Converts team names it seperate categorical columns(team name columns)
final = pd.get_dummies(df2, columns=['team_1', 'team_2'])

In [ ]:
# Features for model
X = final.copy()
X = final.drop('map_winner', axis=1)
# X

In [ ]:
X.shape

In [ ]:
# target map_winner
y = final['map_winner'].values
# y[:5]

### LogisticRegression

In [ ]:
# splitting training and testing data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.80)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, train_size=0.80)

In [ ]:
# training linear logistic regression model
model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)
model.fit(X_train, y_train)

In [ ]:
#Model Accuracy Score
score1 = model.score(X_train, y_train)
score2 = model.score(X_test, y_test)
print(f'train={score1}, test={score2}')

#### output Learning Results

### SVM

In [ ]:
# importing libraries
import numpy as np
from sklearn.svm import SVC
from sklearn import svm
model = svm.SVC()

In [ ]:
# training SVM model
model.fit(X_train, y_train)

In [ ]:
#Model Accuracy Score
score1 = model.score(X_train, y_train)
score2 = model.score(X_test, y_test)
print(f'train={score1}, test={score2}')

#### output SVM model

## Random Forest Classifier

In [ ]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Training Model and displaying model accuracy scores for both training and test
rf = RandomForestClassifier(n_estimators=500, max_depth=60,random_state=0) 
rf.fit(X_train, y_train)

In [ ]:
#Model Accuracy Score
score1 = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)
print(f'train={score1}, test={score2}')

#### output RandomForest Classifier